# Task 2-1 TF2


Compare performance (loss, accuracy, and examples of predictions of trained model) of several implementations for MNIST dataset classification mentioned in the lecture materials: single-layer neural network and Multi-Layer Perceptron (MLP) (in both, TF1 and TF2). But this time, use Fashion-MNIST dataset instead. Play with training hyper-parameters (e.g. network architecture, learning rates, optimizers, etc.) and compare performance… Present comparison results in a table (including corresponding hyper-parameters settings). 

In [35]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten
from tensorflow.python.keras.optimizers import gradient_descent_v2, adam_v2 
import tensorflow as tf
import itertools
import numpy as np
import pandas as pd

In [36]:
(X_train, Y_train), (X_test, Y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [37]:
input_shape = (28, 28)
learning_rate = 0.01

In [38]:
X_train = X_train.astype(np.float32) / 255.0
X_test = X_test.astype(np.float32) / 255.0

In [39]:
single_layer_results_table = pd.DataFrame(columns=['accuracy', 'loss', 'learning_rate', 'neurons', 'epochs', 'batch_size'])
multi_layer_results_table = pd.DataFrame(columns=['accuracy', 'loss', 'learning_rate', 'neurons', 'hidden_layers', 'dropout', 'epochs', 'batch_size'])

In [40]:
def single_layer(neurons, learning_rate):
    model = Sequential([
        Flatten(input_shape=input_shape),
        Dense(neurons, activation='relu', kernel_initializer='he_normal'),
        Dense(10, activation='softmax')
    ])
    
    model.compile(optimizer=adam_v2.Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [41]:
def multi_layer(neurons, hidden_layers, learning_rate, dropout=.5):
    model = Sequential()
    model.add(Flatten(input_shape=input_shape))

    for i in range(hidden_layers):
        model.add(Dense(neurons, activation='relu'))
        model.add(Dropout(dropout))

    model.add(Dense(10, activation='softmax'))
    
    model.compile(optimizer=adam_v2.Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model


In [42]:
def model_training_and_evaluation(*, model, X_train, Y_train, epochs, validation_data, batch_size=16):
    history = model.fit(X_train, Y_train, epochs=epochs, validation_data=validation_data, batch_size=batch_size, verbose=0)
    loss, accuracy = model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=0)

    return history, loss, accuracy

## TF2 Single-layer

In [43]:
tf.compat.v1.enable_v2_behavior()

In [44]:
def single_layer_create_and_evaluate(*, 
                                     learning_rate, 
                                     neurons, 
                                     X_train, 
                                     Y_train, 
                                     epochs, 
                                     validation_data, 
                                     batch_size=16):
    global single_layer_results_table
    model = single_layer(learning_rate=learning_rate, neurons=neurons)
    
    _, loss, accuracy = model_training_and_evaluation(model=model, X_train=X_train, Y_train=Y_train, epochs=epochs, validation_data=validation_data, batch_size=batch_size)
    
    d_row = {
        'accuracy': accuracy, 
        'loss': loss, 
        'learning_rate': learning_rate,
        'neurons': neurons,
        'epochs': epochs,
        'batch_size': batch_size
    }
    
    single_layer_results_table = single_layer_results_table.append(d_row,ignore_index=True)
    
    return single_layer_results_table.iloc[-1:]
    

In [45]:
epochs_list = [2, 10]
batch_sizes = [8, 24]
neurons_list = [4, 16]
learning_rates = [0.1, 0.05]
hidden_layers_list = [0, 3]
dropouts = [0.2, 0.4]

In [46]:
sl_parameters = [
    neurons_list,
    epochs_list,
    learning_rates,
    batch_sizes
]

ml_parameters = [
    neurons_list,
    hidden_layers_list,
    dropouts,
    epochs_list,
    learning_rates,
    batch_sizes
]

In [47]:
for neurons, epochs, learning_rate, batch_size in list(itertools.product(*sl_parameters)):
    single_layer_create_and_evaluate(
        learning_rate=learning_rate,
        neurons=neurons,
        X_train=X_train,
        Y_train=Y_train,
        epochs=epochs,
        validation_data=(X_test, Y_test),
        batch_size=batch_size
    )

TypeError: single_layer() got an unexpected keyword argument 'optimizer'

## TF2 Multi-Layer Perceptron

In [ ]:
def multi_layer_create_and_evaluate(*, 
                                    learning_rate, 
                                    neurons, 
                                    hidden_layers,
                                    dropout=0.5,
                                    X_train, 
                                    Y_train, 
                                    epochs, 
                                    validation_data, 
                                    batch_size=16):
    global multi_layer_results_table
    model = multi_layer(learning_rate=learning_rate, neurons=neurons, hidden_layers=hidden_layers, dropout=dropout)
    
    _, loss, accuracy = model_training_and_evaluation(model=model, X_train=X_train, Y_train=Y_train, epochs=epochs, validation_data=validation_data, batch_size=batch_size)
    
    d_row = {
        'accuracy': accuracy, 
        'loss': loss, 
        'learning_rate': learning_rate,
        'neurons': neurons,
        'hidden_layers': hidden_layers,
        'dropout': dropout,
        'epochs': epochs,
        'batch_size': batch_size
    }
    
    multi_layer_results_table = multi_layer_results_table.append(d_row,ignore_index=True)
    
    return multi_layer_results_table.iloc[-1:]
    

In [ ]:
for neurons, hidden_layers, dropout, epochs, learning_rate, batch_size in list(itertools.product(*ml_parameters)):
    multi_layer_create_and_evaluate(
        learning_rate=learning_rate,
        neurons=neurons,
        hidden_layers=hidden_layers,
        dropout=dropout,
        X_train=X_train,
        Y_train=Y_train,
        epochs=epochs,
        validation_data=(X_test, Y_test),
        batch_size=batch_size
    )

KeyboardInterrupt: 

In [ ]:
single_layer_results_table

In [ ]:
multi_layer_results_table